# Sample of calculating stress on object

In [ ]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np


def bottom(x, on_boundary):
    return (on_boundary and fe.near(x[1], 0.0))

# Функция деформации 
def epsilon(u):
    return fe.sym(fe.grad(u))

# Функция напряжение
def sigma(u):
    return lambda_*fe.div(u)*fe.Identity(2) + 2*mu*epsilon(u)

# --------------------
# Параметры
# --------------------

# Плотность
d = float(input("Плотность: ") or "680.0")
rho = fe.Constant(d)

# Модуль Юнга и коэффициент Пуассона
E = float(input("Модуль Юнга: ") or "0.02e6")
nu = float(input("Коэффициент Пуассона: ") or "0.45")

# Постоянные Ламме
lambda_ = E*nu/(1+nu)/(1-2*nu)
mu = E/2/(1+nu)

# Размерность области
# l_x = float(input("Размерность области l_x: ") or "100.0")
# l_y = float(input("Размерность области l_y: ") or "100.0")

# # Количество элементов
# n_x = float(input("Количество элементов n_x: ") or "50.0")
# n_y = float(input("Количество элементов n_y: ") or "50.0")

# Нагрузка
g_z = float(input("Нагрузка g_z: ") or "-9.8")
b_z = float(input("Нагрузка b_z: ") or "-10.0")
g = fe.Constant((0.0, g_z))
b = fe.Constant((0.0, b_z))

# Модель
model = "plane_strain"
if model == "plane_stress":
    lambda_ = 2*mu*lambda_/(lambda_+2*mu)

# --------------------
# Геометрия
# --------------------
mesh = fe.Mesh("rectangle_mesh.xml")
# mesh = fe.Mesh("circle_mesh.xml")
# mesh = fe.Mesh("domain_mesh.xml")

# Определение области условий Неймана
boundaries = fe.MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)

# top = fe.AutoSubDomain(lambda x: fe.near(x[1], l_y))

# top.mark(boundaries, 1)
ds = fe.ds(subdomain_data=boundaries)

# --------------------
# Функциональные пространства
# --------------------
V = fe.VectorFunctionSpace(mesh, "CG", 1)
u_tr = fe.TrialFunction(V)
u_test = fe.TestFunction(V)

# --------------------
# Граничные условия
# --------------------
bc = fe.DirichletBC(V, fe.Constant((0.0, 0.0)), bottom)

# --------------------
# Слабая форма
# --------------------
a = fe.inner(sigma(u_tr), epsilon(u_test))*fe.dx
l = rho*fe.dot(b, u_test)*fe.dx + fe.inner(g, u_test)*ds(1)

# --------------------
# Вычеслябшая функция
# --------------------
u = fe.Function(V)
A_ass, L_ass = fe.assemble_system(a, l, bc)

fe.solve(A_ass, u.vector(), L_ass)

print(np.amax(u.vector()[:]))

# --------------------
# Последующий графический вывод
# --------------------
fe.plot(u, mode="displacement")
plt.show()